In [1]:
from backtesting import Strategy
from backtesting.lib import crossover
from backtesting import Backtest
import pandas as pd
from backtesting.test import SMA
import pandas as pd
from bokeh.io import export_png
from backtesting.lib import SignalStrategy, TrailingStrategy
from backtesting import Position

/Users/fahad/anaconda3/lib/python3.7/site-packages/backtesting/_plotting.py:34: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `bokeh.io.reset_output()`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [2]:
MAR_predictions = pd.read_csv("/Users/fahad/project_repo/models/final_models/Financial_Analysis/predictions/MAR_predictions.csv", index_col=0)


In [3]:
MAR_predictions.head()

,Date,Predictions
0,2019-07-31,0
1,2019-06-28,0
2,2019-05-31,0
3,2019-04-30,1
4,2019-03-29,0


In [4]:
MAR_predictions.Date = pd.to_datetime(MAR_predictions.Date, format="%Y-%m-%d")

In [5]:
MAR_predictions['Predictions'] = MAR_predictions['Predictions'].shift(-1)


In [6]:
MAR_predictions = MAR_predictions.dropna()


In [7]:
MAR_predictions = MAR_predictions[::-1]

In [8]:
MAR_predictions.head(10)

,Date,Predictions
112,2010-03-31,1.0
111,2010-04-30,0.0
110,2010-05-28,0.0
109,2010-06-30,0.0
108,2010-07-30,1.0
107,2010-08-31,1.0
106,2010-09-30,1.0
105,2010-10-29,1.0
104,2010-11-30,1.0
103,2010-12-31,0.0


In [9]:
MAR_monthly = pd.read_csv("/Users/fahad/project_repo/data/final/MAR/MAR_monthly.csv", index_col =0)

In [10]:
MAR_monthly.columns = ['Open','High','Low','Close','Volume']

In [11]:
MAR_predictions = MAR_predictions.set_index("Date")

In [12]:
MAR_monthly.head()

,Open,High,Low,Close,Volume
2019-08-15,138.99,140.490,124.77,126.70,23829100
2019-07-31,142.41,144.240,135.39,139.06,31330900
2019-06-28,124.76,140.460,123.46,140.29,31391900
2019-05-31,137.80,140.371,122.46,124.84,40716900
2019-04-30,126.24,137.100,125.85,136.42,29052100


In [13]:
MAR_final_df = pd.merge(MAR_predictions, MAR_monthly, right_index=True, left_index=True)

In [14]:
MAR_final_df.index = pd.to_datetime(MAR_final_df.index, format ="%Y-%m-%d")

In [15]:
equity_dict = dict()

In [16]:
class BH(TrailingStrategy):

    
    def init(self):
        
        super().init()
        self.set_trailing_sl(4)
        self.start = self.data.index[0]
        
    def next(self):
        super().init()
        equity_dict[self.data.index[-1]] = TrailingStrategy.equity.__get__(self)
        
        position = Strategy.position.__get__(self)
#         print (type(self.start))
        if self.data.index[-1] > self.start and (position.is_long!=True):
        
            self.buy()
        else:
            pass

In [17]:
bt = Backtest(MAR_final_df, BH, cash=10000, commission=.01)
bt.run()



Start                     2010-03-31 00:00:00
End                       2019-07-31 00:00:00
Duration                   3409 days 00:00:00
Exposure [%]                                0
Equity Final [$]                      37191.6
Equity Peak [$]                         39406
Return [%]                            271.916
Buy & Hold Return [%]                  341.18
Max. Drawdown [%]                    -34.4247
Avg. Drawdown [%]                    -14.3006
Max. Drawdown Duration      818 days 00:00:00
Avg. Drawdown Duration      313 days 00:00:00
# Trades                                    0
Win Rate [%]                              NaN
Best Trade [%]                            NaN
Worst Trade [%]                           NaN
Avg. Trade [%]                            NaN
Max. Trade Duration                       NaT
Avg. Trade Duration                       NaT
Expectancy [%]                            NaN
SQN                                       NaN
Sharpe Ratio                      

In [18]:
bt.plot(superimpose=False, plot_volume=False)

In [19]:
df = pd.DataFrame.from_dict([equity_dict])

In [20]:
df = df.T

In [21]:
df.columns = ['Equity']

In [22]:
export = df.to_csv("/Users/fahad/project_repo/data/final/Equity/BH/MAR_BH_Equity.csv")

In [23]:
df

,Equity
2010-04-30,10000.000000
2010-05-28,8946.194457
2010-06-30,8007.445801
2010-07-30,9069.221347
2010-08-31,8552.240962
2010-09-30,9582.724885
2010-10-29,9909.013592
2010-11-30,10486.705072
2010-12-31,11109.863012
2011-01-31,10561.591005
